# Demo: *Μοντέλο Ευθυγράμμισης Ομιλίας–Κειμένου*

## Οδηγίες Χρήσης Demo

1. **Ανεβάστε ένα αρχείο ήχου ή βίντεο** (π.χ. .wav, .mp3, .mp4).
2. **Ακούστε το αρχείο** για να βεβαιωθείτε ότι είναι σωστό.
3. **Επιλέξτε διάρκεια και επικάλυψη τμημάτων** με sliders.
4. **Επιλέξτε checkpoint** για το μοντέλο ευθυγράμμισης.
5. **Υπολογίστε τα embeddings** για όλα τα segments (κουμπί & μπάρα προόδου).
6. **Υποβάλετε ερώτημα** στο πεδίο κειμένου.
7. **Δείτε τα top-k αποτελέσματα** (segments με μεγαλύτερη ομοιότητα, playable).

---

In [4]:
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
import librosa
import numpy as np
import torch
import os
from transformers import AutoModel, AutoFeatureExtractor
from sentence_transformers import SentenceTransformer

# Helper: Load audio (wav/mp3/mp4)
def load_audio(file, target_sr=16000):
    # librosa supports most audio/video formats via ffmpeg
    import tempfile
    with tempfile.NamedTemporaryFile(delete=False) as tmp:
        tmp.write(file['content'])
        tmp_path = tmp.name
    y, sr = librosa.load(tmp_path, sr=target_sr)
    os.remove(tmp_path)
    return y, sr

# Widget: File upload
audio_uploader = widgets.FileUpload(accept='.wav,.mp3,.mp4,.flac,.ogg,.mkv,.avi,.mov', multiple=False)
audio_out = widgets.Output()

# Display upload widget
display(widgets.HTML('<b>1. Ανεβάστε αρχείο ήχου/βίντεο:</b>'))
display(audio_uploader)
display(audio_out)

def on_upload_change(change):
    audio_out.clear_output()
    if audio_uploader.value:
        file_info = next(iter(audio_uploader.value.values()))
        with audio_out:
            y, sr = load_audio(file_info, target_sr=16000)
            display(Audio(y, rate=sr))
            print(f'Διάρκεια: {len(y)/sr:.2f} sec, SR: {sr}')
            global uploaded_audio, uploaded_sr
            uploaded_audio, uploaded_sr = y, sr

audio_uploader.observe(on_upload_change, names='value')

HTML(value='<b>1. Ανεβάστε αρχείο ήχου/βίντεο:</b>')

FileUpload(value=(), accept='.wav,.mp3,.mp4,.flac,.ogg,.mkv,.avi,.mov', description='Upload')

Output()

AttributeError: 'tuple' object has no attribute 'values'

AttributeError: 'tuple' object has no attribute 'values'

---

### 2. Επιλογή παραμέτρων τμηματοποίησης

- Επιλέξτε **διάρκεια τμήματος** (σε δευτερόλεπτα)
- Επιλέξτε **επικάλυψη** (σε δευτερόλεπτα)


In [ ]:
# Sliders for segment duration and overlap
segment_duration_slider = widgets.FloatSlider(
    value=2.0, min=0.5, max=10.0, step=0.1,
    description='Διάρκεια (sec):', continuous_update=False
)
overlap_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=5.0, step=0.1,
    description='Επικάλυψη (sec):', continuous_update=False
)
display(segment_duration_slider, overlap_slider)

# Helper: Segment audio
def segment_audio(y, sr, duration, overlap):
    seg_len = int(duration * sr)
    step = int((duration - overlap) * sr)
    segments = []
    for start in range(0, max(1, len(y) - seg_len + 1), step):
        end = start + seg_len
        if end > len(y):
            break
        segments.append(y[start:end])
    return segments


KeyboardInterrupt: 

---

### 3. Επιλογή Checkpoint & Υπολογισμός Embeddings

- Επιλέξτε checkpoint (μοντέλο ευθυγράμμισης)
- Πατήστε το κουμπί για υπολογισμό embeddings (πρόοδος)


In [ ]:
# Find available checkpoints
def find_checkpoints(logs_dir='logs'):
    ckpts = []
    for root, dirs, files in os.walk(logs_dir):
        for f in files:
            if f.endswith('.ckpt'):
                ckpts.append(os.path.join(root, f))
    return ckpts

ckpt_list = find_checkpoints()
ckpt_dropdown = widgets.Dropdown(
    options=ckpt_list,
    description='Checkpoint:',
    layout=widgets.Layout(width='80%')
)
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Πρόοδος:')
embed_btn = widgets.Button(description='Υπολογισμός Embeddings', button_style='success')
embed_out = widgets.Output()
display(ckpt_dropdown, embed_btn, progress, embed_out)

# Load models (Whisper encoder, Aligner, E5)
whisper_id = 'openai/whisper-tiny'
e5_id = 'intfloat/multilingual-e5-small'
feature_extractor = AutoFeatureExtractor.from_pretrained(whisper_id)
whisper_model = AutoModel.from_pretrained(whisper_id)
e5_model = SentenceTransformer(e5_id)

def compute_segment_embeddings(_):
    embed_out.clear_output()
    if 'uploaded_audio' not in globals():
        with embed_out:
            print('Ανεβάστε πρώτα αρχείο ήχου.')
        return
    y, sr = uploaded_audio, uploaded_sr
    duration = segment_duration_slider.value
    overlap = overlap_slider.value
    segments = segment_audio(y, sr, duration, overlap)
    with embed_out:
        print(f'Τμήματα: {len(segments)}')
    # Load aligner
    ckpt_path = ckpt_dropdown.value
    config_path = os.path.join(os.path.dirname(ckpt_path), 'hparams.yaml')
    import yaml
    with open(config_path) as f:
        config = yaml.safe_load(f)
    from models import get_adapter_class, AlignmentModel
    AdapterClass = get_adapter_class(config.get('adapter-type', 'cnn'))
    adapter = AdapterClass(**config.get('kwargs', {}))
    aligner = AlignmentModel(adapter=adapter, init_tau=config.get('init_tau', 0.07))
    state = torch.load(ckpt_path, map_location='cpu')
    state_dict = state['state_dict'] if 'state_dict' in state else state['model_state_dict']
    if any(k.startswith('model.') for k in state_dict):
        state_dict = {k.replace('model.', '', 1): v for k, v in state_dict.items()}
    aligner.load_state_dict(state_dict, strict=False)
    aligner.eval()
    # Encode segments
    embs = []
    for i, seg in enumerate(segments):
        progress.value = i / len(segments)
        inputs = feature_extractor(seg, sampling_rate=sr, return_tensors='pt', padding='max_length', truncation=True, return_attention_mask=True)
        with torch.no_grad():
            enc_out = whisper_model.encoder(input_features=inputs.input_features, attention_mask=inputs.attention_mask)
        speech_emb = enc_out.last_hidden_state
        attn_mask = inputs.attention_mask
        emb = aligner.encode(speech_emb, attn_mask)
        embs.append(emb.squeeze(0).cpu().numpy())
    progress.value = 1.0
    global segment_embeddings, segment_audio_list
    segment_embeddings = np.stack(embs)
    segment_audio_list = segments
    with embed_out:
        print('Ολοκληρώθηκε ο υπολογισμός των embeddings.')

embed_btn.on_click(compute_segment_embeddings)

---

### 4. Υποβολή Ερωτήματος & Αναζήτηση

- Εισάγετε το ερώτημά σας (σε οποιαδήποτε γλώσσα)
- Επιλέξτε πόσα αποτελέσματα (top-k) να εμφανιστούν
- Πατήστε το κουμπί για αναζήτηση


In [ ]:
# Query widgets
query_box = widgets.Textarea(
    value='',
    placeholder='Γράψτε το ερώτημά σας εδώ...',
    description='Ερώτημα:',
    layout=widgets.Layout(width='80%', height='60px')
)
topk_slider = widgets.IntSlider(
    value=3, min=1, max=10, step=1,
    description='Top-k:', continuous_update=False
)
search_btn = widgets.Button(description='Αναζήτηση', button_style='info')
results_out = widgets.Output()
display(query_box, topk_slider, search_btn, results_out)

# Search logic
def search_segments(_):
    results_out.clear_output()
    if 'segment_embeddings' not in globals():
        with results_out:
            print('Υπολογίστε πρώτα τα embeddings.')
        return
    query = query_box.value.strip()
    if not query:
        with results_out:
            print('Γράψτε ερώτημα.')
        return
    # Encode query
    q_emb = e5_model.encode([f'query: {query}'], normalize_embeddings=True)
    # Compute similarity
    sims = np.dot(segment_embeddings, q_emb[0])
    topk = topk_slider.value
    idxs = np.argsort(sims)[::-1][:topk]
    with results_out:
        for i, idx in enumerate(idxs):
            display(widgets.HTML(f'<b>#{i+1} (score={sims[idx]:.3f})</b>'))
            display(Audio(segment_audio_list[idx], rate=uploaded_sr))

search_btn.on_click(search_segments)